<a href="https://colab.research.google.com/github/Aditya6122/colab-notebooks/blob/main/preprocessing_for_fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
raw_data_path = '/content/drive/MyDrive/fakenews/train.csv'
destination_folder = '/content/drive/MyDrive/fakenews/preprocessed'

In [65]:
import pandas as pd

In [66]:
data = pd.read_csv(raw_data_path)

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [68]:
data = data[['title','text','label']]

In [69]:
data.isna().sum()

title    558
text      39
label      0
dtype: int64

In [70]:
data.dropna(inplace=True)

In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20203 entries, 0 to 20799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20203 non-null  object
 1   text    20203 non-null  object
 2   label   20203 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 631.3+ KB


In [72]:
data.label.value_counts()

0    10387
1     9816
Name: label, dtype: int64

In [73]:
data.head()

,title,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [74]:
import re
import string

In [75]:
def remove_unwanted_char(s):
    s = s.replace('\n',' ')
    s = s.replace('\t',' ')
    s = s.lower()
    chars = []
    for char in s:
        if((ord(char)<97 or ord(char)>122) and not(ord(char)>=48 and ord(char)<=57)):
            chars.append(' ')
        elif(ord(char)>=48 and ord(char)<=57):
            chars.append('')
        else:
            chars.append(char)
    s = ''.join([char for char in chars])
    s = re.sub(' +', ' ', s)
    s = s.lstrip()
    s = s.rstrip()
    return s

In [76]:
data['title'] = data['title'].apply(lambda x : remove_unwanted_char(x))
data['text'] = data['text'].apply(lambda x : remove_unwanted_char(x))

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
X_temp, X_test, y_temp, y_test= train_test_split(data[['title','text']], \
                                                      data['label'], \
                                                      test_size=0.1,\
                                                      shuffle=True, \
                                                      stratify = data['label'],\
                                                      random_state=0)

In [79]:
X_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18182 entries, 12010 to 19445
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   18182 non-null  object
 1   text    18182 non-null  object
dtypes: object(2)
memory usage: 426.1+ KB


In [80]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2021 entries, 17291 to 11580
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2021 non-null   object
 1   text    2021 non-null   object
dtypes: object(2)
memory usage: 47.4+ KB


In [81]:
X_train, X_valid, y_train, y_valid= train_test_split(X_temp[['title','text']], \
                                                      y_temp, \
                                                      test_size=0.2,\
                                                      stratify = y_temp,\
                                                      random_state=0)

In [82]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14545 entries, 13199 to 15284
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   14545 non-null  object
 1   text    14545 non-null  object
dtypes: object(2)
memory usage: 340.9+ KB


In [30]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3637 entries, 6989 to 4353
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   3637 non-null   object
 1   text    3637 non-null   object
dtypes: object(2)
memory usage: 85.2+ KB


In [106]:
train_set = pd.concat([X_train,y_train],axis=1)
valid_set = pd.concat([X_valid,y_valid],axis=1)
test_set = pd.concat([X_test,y_test],axis=1)

In [107]:
train_set.reset_index(inplace=True)
valid_set.reset_index(inplace=True)
test_set.reset_index(inplace=True)

In [108]:
train_set.drop('index',inplace=True,axis=1)
valid_set.drop('index',inplace=True,axis=1)
test_set.drop('index',inplace=True,axis=1)

In [109]:
train_set.to_csv('/content/drive/MyDrive/fakenews/preprocessed/train.csv',index=False)
test_set.to_csv('/content/drive/MyDrive/fakenews/preprocessed/test.csv',index=False)
valid_set.to_csv('/content/drive/MyDrive/fakenews/preprocessed/valid.csv',index=False)

In [127]:
train_data = pd.read_csv('/content/drive/MyDrive/fakenews/preprocessed/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/fakenews/preprocessed/test.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/fakenews/preprocessed/valid.csv')

In [128]:
train_data.isna().sum()

title    0
text     0
label    0
dtype: int64

In [129]:
test_data.isna().sum()

title    0
text     0
label    0
dtype: int64

In [130]:
valid_data.isna().sum()

title    0
text     0
label    0
dtype: int64

In [114]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)
valid_data.dropna(inplace=True)

In [115]:
train_data.reset_index(inplace=True)
valid_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [116]:
train_data.drop('index',inplace=True,axis=1)
valid_data.drop('index',inplace=True,axis=1)
test_data.drop('index',inplace=True,axis=1)

In [120]:
train_data.isna().sum()

title    0
text     0
label    0
dtype: int64

In [121]:
valid_data.isna().sum()

title    0
text     0
label    0
dtype: int64

In [122]:
test_data.isna().sum()

title    0
text     0
label    0
dtype: int64

In [123]:
train_data.to_csv('/content/drive/MyDrive/fakenews/preprocessed/train.csv',index=False)
test_data.to_csv('/content/drive/MyDrive/fakenews/preprocessed/test.csv',index=False)
valid_data.to_csv('/content/drive/MyDrive/fakenews/preprocessed/valid.csv',index=False)

In [21]:
raw_data_path = '/content/drive/MyDrive/fakenews/train.csv'
destination_folder = '/content/drive/MyDrive/fakenews/preprocessed'

train_test_ratio = 0.10
train_valid_ratio = 0.80

first_n_words = 200

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
def trim_string(x):

    x = x.split(maxsplit=first_n_words)
    x = ' '.join(x[:first_n_words])

    return x

In [ ]:
# Read raw data
df_raw = pd.read_csv(raw_data_path)

# Prepare columns
df_raw['label'] = (df_raw['label'] == 'FAKE').astype('int')
df_raw['titletext'] = df_raw['title'] + ". " + df_raw['text']
df_raw = df_raw.reindex(columns=['label', 'title', 'text', 'titletext'])

# Drop rows with empty text
df_raw.drop( df_raw[df_raw.text.str.len() < 5].index, inplace=True)

# Trim text and titletext to first_n_words
df_raw['text'] = df_raw['text'].apply(trim_string)
df_raw['titletext'] = df_raw['titletext'].apply(trim_string) 

# Split according to label
df_real = df_raw[df_raw['label'] == 0]
df_fake = df_raw[df_raw['label'] == 1]

# Train-test split
df_real_full_train, df_real_test = train_test_split(df_real, train_size = train_test_ratio, random_state = 1)
df_fake_full_train, df_fake_test = train_test_split(df_fake, train_size = train_test_ratio, random_state = 1)

# Train-valid split
df_real_train, df_real_valid = train_test_split(df_real_full_train, train_size = train_valid_ratio, random_state = 1)
df_fake_train, df_fake_valid = train_test_split(df_fake_full_train, train_size = train_valid_ratio, random_state = 1)

# Concatenate splits of different labels
df_train = pd.concat([df_real_train, df_fake_train], ignore_index=True, sort=False)
df_valid = pd.concat([df_real_valid, df_fake_valid], ignore_index=True, sort=False)
df_test = pd.concat([df_real_test, df_fake_test], ignore_index=True, sort=False)

# Write preprocessed data
df_train.to_csv(destination_folder + '/train.csv', index=False)
df_valid.to_csv(destination_folder + '/valid.csv', index=False)
df_test.to_csv(destination_folder + '/test.csv', index=False)